# <span style="color: orange;">Preprocessing Démographie et Requêtes Google</span>


### <span style="color: green;">Import des librairies</span>

In [22]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import unicodedata
import glob

import sys
sys.path.append("../src")
from preprocessing import normalize_region_name, get_population_data, consolidate_trends_to_single_sheet, clean_google_trends_data

### <span style="color: green;">Chargement des données</span>

In [23]:
# Train et Test datasets
df_train = pd.read_csv('../data/raw/train.csv')
df_test = pd.read_csv('../data/raw/test.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           9196 non-null   int64 
 1   week         9196 non-null   int64 
 2   region_code  9196 non-null   int64 
 3   region_name  9196 non-null   object
 4   TauxGrippe   9196 non-null   int64 
dtypes: int64(4), object(1)
memory usage: 359.3+ KB


In [24]:
# Données démographiques
filepath = '../data/raw/estim-pop-areg-sexe-gca-1975-2015.xls'
df_pop_train = get_population_data(filepath, 2004, 2011)
df_pop_test = get_population_data(filepath, 2012, 2013)
print(f" Il y a {df_pop_train['region'].nunique()} régions sélectionnées dans les données démographiques.")

Aperçu des données extraites (2004-2011) :


,region,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,year
12,Languedoc-Roussillon,622980,619529,715033,435841,276663,2670046,2011
9,Franche-Comté,290000,286318,316818,173479,106825,1173440,2011
21,Rhône-Alpes,1593489,1620992,1670893,870502,527665,6283541,2011
158,Bourgogne,381173,395993,451439,236198,157653,1622456,2004
78,Languedoc-Roussillon,604415,619124,701788,396198,260193,2581718,2008


Aperçu des données extraites (2012-2013) :


,region,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,year
43,Rhône-Alpes,1605255,1620387,1686929,892413,536176,6341160,2012
12,Languedoc-Roussillon,629807,618769,729438,466336,285371,2729721,2013
6,Centre,614810,577670,695917,414502,267649,2570548,2013
15,Midi-Pyrénées,670855,702515,801955,464448,314384,2954157,2013
40,Picardie,496904,476397,526911,267570,154560,1922342,2012


 Il y a 22 régions sélectionnées dans les données démographiques.


In [25]:
# Données Requêtes Google
# Récupération des fichiers CSV de requêtes Google par région
all_csvs = glob.glob("../data/raw/RequetesGoogleParRegion/*.csv")
print(f" Il y a {len(all_csvs)} fichiers de requêtes Google par région.")

 Il y a 22 fichiers de requêtes Google par région.


### <span style="color: green;">Preprocessing</span>

#### <span style="color: green;">Données démographiques</span>

In [26]:
df_pop_train['region_normalized'] = df_pop_train['region'].apply(normalize_region_name)
df_pop_test['region_normalized'] = df_pop_test['region'].apply(normalize_region_name)
# Vérification
display(df_pop_train[['region', 'region_normalized']].sample(5))

,region,region_normalized
52,Corse,CORSE
102,Lorraine,LORRAINE
128,Picardie,PICARDIE
111,Aquitaine,AQUITAINE
12,Languedoc-Roussillon,LANGUEDOCROUSSILLON


In [27]:
# Chemins des fichiers Excel
output_csv_train = '../data/processed/pop_train.csv'
output_csv_test = '../data/processed/pop_test.csv'
# Export vers CSV
df_pop_train.to_csv(output_csv_train, index=False)
df_pop_test.to_csv(output_csv_test, index=False)

#### <span style="color: green;">Données requêtes Google</span>

In [28]:
# Consolidation des données Google Trends en 1 seul fichier Excel
output_excel = '../data/processed/google_trends_consolide.xlsx'
# Création du fichier Excel des requetes avec plusieurs feuilles
with pd.ExcelWriter(output_excel) as writer:
    for file in all_csvs:
        # On récupère le nom de la région depuis le nom du fichier
        region_raw = os.path.splitext(os.path.basename(file))[0]    
        # On normalise ce nom pour la feuille Excel
        sheet_name = normalize_region_name(region_raw)
        # Ajout : Renommage spécifique pour CentreValdeLoire
        if sheet_name == "CENTREVALDELOIRE":
            sheet_name = "CENTRE"    
        # Lecture du CSV
        # header=2 signifie qu'on utilise la 3ème ligne comme en-tête (on saute les lignes 0 et 1)
        df = pd.read_csv(file, header=1)        
        # Écriture dans la feuille correspondante
        df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Ajouté : {region_raw} -> Feuille : {sheet_name}")
print(80*'-')
print(f"Le fichier '{output_excel}' a été créé.")

Ajouté : Alsace -> Feuille : ALSACE
Ajouté : Aquitaine -> Feuille : AQUITAINE
Ajouté : Auvergne -> Feuille : AUVERGNE
Ajouté : BasseNormandie -> Feuille : BASSENORMANDIE
Ajouté : Bourgogne -> Feuille : BOURGOGNE
Ajouté : Bretagne -> Feuille : BRETAGNE
Ajouté : CentreValdeLoire -> Feuille : CENTRE
Ajouté : ChampagneArdenne -> Feuille : CHAMPAGNEARDENNE
Ajouté : Corse -> Feuille : CORSE
Ajouté : FrancheComte -> Feuille : FRANCHECOMTE
Ajouté : HauteNormandie -> Feuille : HAUTENORMANDIE
Ajouté : IledeFrance -> Feuille : ILEDEFRANCE
Ajouté : LanguedocRoussillon -> Feuille : LANGUEDOCROUSSILLON
Ajouté : Limousin -> Feuille : LIMOUSIN
Ajouté : Lorraine -> Feuille : LORRAINE
Ajouté : MidiPyrenees -> Feuille : MIDIPYRENEES
Ajouté : NordPasdeCalais -> Feuille : NORDPASDECALAIS
Ajouté : PaysdelaLoire -> Feuille : PAYSDELALOIRE
Ajouté : Picardie -> Feuille : PICARDIE
Ajouté : PoitouCharentes -> Feuille : POITOUCHARENTES
Ajouté : ProvenceAlpesCotedAzur -> Feuille : PROVENCEALPESCOTEDAZUR
Ajouté : R

In [29]:
# Cleaning des données Google Trends consolidées
input_excel = '../data/processed/google_trends_consolide.xlsx'
output_excel = '../data/processed/google_trends_clean.xlsx'
clean_google_trends_data(input_excel, output_excel) 

Traitement de 22 feuilles...
Succès ! Fichier nettoyé créé : ../data/processed/google_trends_clean.xlsx


In [30]:
# Consolidation des données Google Trends nettoyées en un seul DataFrame
input_clean = '../data/processed/google_trends_clean.xlsx'
output_final = '../data/processed/google_trends_requetes.xlsx'
df_requetes = consolidate_trends_to_single_sheet(input_clean, output_final)

Lecture du fichier : ../data/processed/google_trends_clean.xlsx
Consolidation de 22 feuilles...
Succès ! Fichier unique créé : ../data/processed/google_trends_requetes.xlsx
Dimensions finales : (3388, 6)


#### <span style="color: green;">Données du train et test</span>

In [31]:
# Création de la colonne 'year'
df_train['year'] = df_train['week'].astype(str).str[:4].astype(int)
df_test['year'] = df_test['week'].astype(str).str[:4].astype(int)
# Création de la colonne 'month' au format datetime (1er jour du mois)
df_train['month'] = pd.to_datetime(df_train['week'].astype(str) + '1', format='%Y%W%w').dt.to_period('M').dt.to_timestamp()
df_test['month'] = pd.to_datetime(df_test['week'].astype(str) + '1', format='%Y%W%w').dt.to_period('M').dt.to_timestamp()
# Vérification
display(df_train[['week', 'month', 'year']].sample(5))

,week,month,year
922,201111,2011-03-01,2011
1365,201042,2010-10-01,2010
7966,200503,2005-01-01,2005
1604,201032,2010-08-01,2010
1005,201107,2011-02-01,2011


In [32]:
# Normalisation des noms de régions
df_train['region_normalized'] = df_train['region_name'].apply(normalize_region_name)
df_test['region_normalized'] = df_test['region_name'].apply(normalize_region_name)
# Vérification
display(df_train[['region_name', 'region_normalized']].sample(5))

,region_name,region_normalized
3210,ILE-DE-FRANCE,ILEDEFRANCE
9039,PROVENCE-ALPES-COTE-D-AZUR,PROVENCEALPESCOTEDAZUR
8550,MIDI-PYRENEES,MIDIPYRENEES
2274,CORSE,CORSE
5160,LIMOUSIN,LIMOUSIN


### <span style="color: green;">Fusions finales</span>

- On fusionne `df_train`et `df_train` avec `df_pop_train`et `df_pop_test`

In [33]:
# Fusion left join du df_train et df_pop
df_merged_train = pd.merge(df_train, df_pop_train, on=['year', 'region_normalized'], how='left')
df_merged_test = pd.merge(df_test, df_pop_test, on=['year', 'region_normalized'], how='left')
print("Shape après fusion :", df_merged_train.shape)
display(df_merged_train.sample(5))

Shape après fusion : (9196, 15)


,Id,week,region_code,region_name,TauxGrippe,year,month,region_normalized,region,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total
5968,11491,200642,24,CENTRE,0,2006,2006-10-01,CENTRE,Centre,613729,621338,703346,343532,237622,2519567
3829,9352,200835,72,AQUITAINE,0,2008,2008-09-01,AQUITAINE,Aquitaine,719728,768406,892952,469806,326733,3177625
17,5540,201152,22,PICARDIE,61,2011,2011-12-01,PICARDIE,Picardie,499506,481778,526267,259503,151101,1918155
1006,6529,201107,52,PAYS-DE-LA-LOIRE,102,2011,2011-02-01,PAYSDELALOIRE,Pays de la Loire,918142,879018,954546,513503,335904,3601113
5240,10763,200723,26,BOURGOGNE,0,2007,2007-06-01,BOURGOGNE,Bourgogne,377107,383123,462949,240525,170187,1633891


- On rajoute `df_requetes`

In [34]:
df_merged_train = pd.merge(df_merged_train, df_requetes, on=['month', 'region_normalized'], how='left')
df_merged_test = pd.merge(df_merged_test, df_requetes, on=['month', 'region_normalized'], how='left')
print("Shape après fusion avec requêtes :", df_merged_train.shape)
display(df_merged_train.sample(5))

Shape après fusion avec requêtes : (9196, 19)


,Id,week,region_code,region_name,TauxGrippe,year,month,region_normalized,region,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,Mois,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
4438,9961,200808,52,PAYS-DE-LA-LOIRE,270,2008,2008-02-01,PAYSDELALOIRE,Pays de la Loire,898680,886732,948464,464290,312004,3510170,2008-02,7,5,5
7077,12600,200544,31,NORD-PAS-DE-CALAIS,0,2005,2005-10-01,NORDPASDECALAIS,Nord - Pas-de-Calais,1119560,1104070,1067337,444839,279807,4015613,2005-10,35,9,8
8822,14345,200417,42,ALSACE,0,2004,2004-04-01,ALSACE,Alsace,453398,508491,492030,222857,114369,1791145,2004-04,2,2,2
3220,8743,200911,94,CORSE,34,2009,2009-03-01,CORSE,Corse,62752,74505,87688,50353,30376,305674,2009-03,23,0,0
7198,12721,200538,26,BOURGOGNE,23,2005,2005-09-01,BOURGOGNE,Bourgogne,379809,390736,457639,235500,162506,1626190,2005-09,7,2,2


### <span style="color: green;">Nettoyages finaux</span>

In [35]:
df_merged_train.columns

Index(['Id', 'week', 'region_code', 'region_name', 'TauxGrippe', 'year',
       'month', 'region_normalized', 'region', 'pop_0_19', 'pop_20_39',
       'pop_40_59', 'pop_60_74', 'pop_75_plus', 'pop_total', 'Mois',
       'requete_grippe', 'requete_grippe_aviaire_vaccin',
       'requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie'],
      dtype='object')

- On ne selectionne que les colonnes 'utiles'

In [36]:
# Liste des colonnes à conserver
keep_cols = ['Id', 'week', 'region_name', 'TauxGrippe',   # Les données du dataset central cible
             'pop_0_19', 'pop_20_39', 'pop_40_59', 'pop_60_74', 'pop_75_plus', 'pop_total', # Données démographiques
             'requete_grippe', 'requete_grippe_aviaire_vaccin', 'requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie' # Données Google Trends
             # Données météo à ajouter plus tard
]
# Sélection
df_train = df_merged_train[keep_cols].copy()
df_test = df_merged_test[[c for c in keep_cols if c != 'TauxGrippe']].copy()
# Affichage pour vérification
print(f"Dimensions : \n Train : {df_train.shape} \n Test : {df_test.shape}")
display(df_train.head(), df_test.head())

Dimensions : 
 Train : (9196, 13) 
 Test : (2288, 12)


,Id,week,region_name,TauxGrippe,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
0,5523,201152,ALSACE,66,448112,479905,526481,250849,146978,1852325,2,2,2
1,5524,201152,AQUITAINE,24,732120,765143,896030,516065,344875,3254233,2,2,2
2,5525,201152,AUVERGNE,91,294709,305778,375485,223830,150880,1350682,3,3,2
3,5526,201152,BASSE-NORMANDIE,49,354514,336201,401766,228888,154315,1475684,2,2,2
4,5527,201152,BOURGOGNE,33,372023,369737,450664,269109,181201,1642734,2,2,1


,Id,week,region_name,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
0,3235,201352,ALSACE,446622,471708,529960,266350,153543,1868183,2,2,2
1,3236,201352,AQUITAINE,742922,761895,911307,547389,353376,3316889,3,2,2
2,3237,201352,AUVERGNE,294350,299336,375266,235294,153422,1357668,4,3,3
3,3238,201352,BASSE-NORMANDIE,352006,325231,400270,242425,158780,1478712,2,2,2
4,3239,201352,BOURGOGNE,370747,357632,447520,282940,183848,1642687,3,2,2


### <span style="color: green;">Export CSV</span>

In [37]:
df_train.to_csv('../data/processed/train.csv', index=False)
df_test.to_csv('../data/processed/test.csv', index=False)

### <span style="color: green;">Vérification des datasets</span>

- On a réussi à avoir un fichier avec les données démographiques et un fichier avec les données de requetes Google.
- Ces deux fichiers ont des informations pertinentes et avec des clés appropriées pour les jointures.
- On les a joint au train et test de base.

### <span style="color: green;">Fin</span>